# An introduction to visualization with yt

[`yt`](https://yt-project.org) is a Python toolkit that provides a generic interface for analysis and visualization of spatially-organised data, with built-in compatibility for various data formats (Athena++, RAMSES, AREPO, to name a few).
It supports grid-based data (regular, streched, and various flavours of AMR), as well as particle data (point-like and SPH).
Its key features are:
- a code-agnostic interface (specificities of data formats are rarely relevant)
- dimensionally meaningful analysis
- lazy data loading (only load in memory what's actually needed, when it's needed)
- parallel capacity (with `mpi4py`)


In this tutorial, we'll demonstrate how to load data, and produce simple visulations with yt in a couple lines of Python, and show how to go beyond the defaults.
We will cover the following basic functionalities.
- `yt.load`
- `yt.SlicePlot`
- `yt.ProjectionPlot`
- `yt.Dataset.add_field`

Then, we will see how to enable compatibility with Idefix (and Pluto !) data files using the [`yt_idefix`](https://github.com/neutrinoceros/yt_idefix) extension.


For completness, `yt` supports many operations that will not be covered here, like
- plotting multiple fields with a single `SlicePlot` object
- data reduction to 1D-profile
- "phase plots"
- volume rendering
- off-axis slices and projections
- region selection
- exporting reduced datasets to `HDF5`

*note* that many of these features may have only partial support for non-cartesian geometries.

Explore [the documentation](https://yt-project.org/doc/) and [the cookbook](https://yt-project.org/doc/cookbook/index.html) for much more.


In [ ]:
import yt

In [ ]:
ds = yt.load_sample("IsolatedGalaxy")
# ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy")

In [ ]:
yt.SlicePlot(ds, "z", ("gas", "density"))

In [ ]:
# plot the *column* density
yt.ProjectionPlot(ds, "z", ("gas", "density"))

In [ ]:
p = yt.SlicePlot(ds, "z", ("gas", "density"))
p

In [ ]:
p.zoom(30)

In [ ]:
p.annotate_timestamp(draw_inset_box=True)

In [ ]:
p.annotate_streamlines(("gas", "velocity_x"), ("gas", "velocity_y"), color="white")

In [ ]:
p.swap_axes()

In [ ]:
p.flip_horizontal()

## Inspecting and defining data fields

`yt` has a notion of 'native' VS 'derived' data fields. The former are directly read from disk, while the latter are computed on the fly.
Many fields are often available out-of-the-box



In [ ]:
import yt

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy")
ds.field_list

But it's always possible to define more fields at runtime !
Let's see how we would define a simple field representing the gravitational pull of central potential created by $10^6$ solar masses on an Earth-mass test particle as a function of position.

In `yt`, data fields are essentially [*numpy arrays with units*](https://unyt.readthedocs.io/en/stable/). Working with physically meaningful quantities allows `yt` to verify dimensional consistency at runtime. This crucial feature is powered by the `unyt` library. Let's use it here to define the $GM$ constant.

In [ ]:
import unyt as un
import unyt.physical_constants as cst

GM = cst.G * (10**6 * cst.Msun)

# visual dimension analysis
(GM * cst.Mearth / un.AU**2).units.dimensions

In [ ]:
# this cell has a bug, can you spot it ?


def gravitational_pull(field, data):
    # the pull exerted by a central mass of 10**6 Msun on a test mass (Mearth)
    # as a function of the position
    xc, yc, zc = data.ds.domain_center
    length_unit = data["index", "x"].units
    d2 = (
        (data["index", "x"] - xc) ** 2
        + (data["index", "y"] - yc) ** 2
        + (data["index", "z"] - zc) ** 2
        + 1e-16 * length_unit**2  # smoothing around origin
    )
    return GM * un.Mearth / d2**2


ds.add_field(
    ("gas", "gravitational_pull"),
    function=gravitational_pull,
    sampling_type="cell",
    units="N",  # enable dimensional analysis check at definition time
    force_override=True,  # make this cell re-runable
)

yt.SlicePlot(ds, "x", ("gas", "gravitational_pull"))

## 2 - Working with idefix data
We'll need to install the `yt_idefix` extension to get compatibility with idefix (and Pluto) data.

In [ ]:
%pip install yt_idefix

In [ ]:
ds = yt.load("../../../idefix/test/HD/FargoPlanet/data.0001.vtk")
p = yt.ProjectionPlot(ds, "z", ("gas", "density"))
p.annotate_title("Default behaviour")

Default units are weird... this is because Idefix doesn't have a concept of runtime units. We can provide them at post-processing time as

In [ ]:
ds = yt.load(
    "../../../idefix/test/HD/FargoPlanet/data.0001.vtk",
    units_override={
        "length_unit": (5.5, "AU"),
        "mass_unit": (1, "Msun"),
    },
)
p = yt.ProjectionPlot(ds, "z", ("gas", "density"))
p.annotate_title("with units override")

... or we can "hide" units (they are still used internally)

In [ ]:
ds = yt.load(
    "../../../idefix/test/HD/FargoPlanet/data.0001.vtk",
    unit_system="code",
)
p = yt.ProjectionPlot(ds, "z", ("gas", "density"))
p.annotate_title("with 'code' unit system")